In [ ]:
import numpy as np
import chipwhisperer as cw
import os
import matplotlib.pylab as plt

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = "d:/data_store"

In [ ]:
mixergroup1, mixergroup2, N = nptload("opticalisolator_mixer_bothclks_120mhz_sync_120msps_ttest_10k")
shuntgroup1, shuntgroup2, N = nptload("baseline120mhz_sync_120msps_ttest_10k")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12,8))

fig.suptitle(r"Mixer PM-SCA (from Clock Measurement via Optical Isolator) vs. Shunt PA-SCA", fontsize=16)

start = 500
end = 3500

x = [1.0/120E6 * i * 1E6 for i in range(0, len(mixergroup1[0]))][start:end]

ax[0,0].plot(x, mixergroup1[0][start:end], 'g')
ax[0,0].set_title("Mixer (Optical Isolator), Single Trace, 120 MS/s", fontsize=14)
ax[0,0].set_ylabel("Avg", fontsize=13)
ax[0,0].set_xlabel(r"Time from Trigger ($\mu$S)", fontsize=13)


ax[0,1].plot(x, np.mean(mixergroup1, axis=0)[start:end], 'g')
ax[0,1].set_title("Mixer (Optical Isolator), Avg. (n=10000)", fontsize=14)
ax[0,1].set_ylabel("Value", fontsize=13)
ax[0,1].set_xlabel(r"Time from Trigger ($\mu$S)", fontsize=13)

ax[1,0].plot(x, shuntgroup1[0][start:end])
ax[1,0].set_title("Shunt Resistor, Single Trace, 120 MS/s", fontsize=14)
ax[1,0].set_ylabel("Avg", fontsize=13)
ax[1,0].set_xlabel(r"Time from Trigger ($\mu$S)", fontsize=13)


ax[1,1].plot(x, np.mean(shuntgroup1, axis=0)[start:end])
ax[1,1].set_title("Shunt Resistor, Avg. (n=10000)", fontsize=14)
ax[1,1].set_ylabel("Value", fontsize=13)
ax[1,1].set_xlabel(r"Time from Trigger ($\mu$S)", fontsize=13)

plt.tight_layout()

plt.savefig("opticalpmwaveform.png", dpi=300)
plt.savefig("opticalpmwaveform.eps", dpi=300)
plt.show()

In [ ]:
import scared
import estraces
import numpy as np

In [ ]:
knownkey = [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]

In [ ]:
def find_success(att, knownkey):
    recovered_masterkey = np.argmax(att.scores, axis=0).astype('uint8')
    #print(recovered_masterkey == knownkey)
    
    for i in range(0, np.shape(att.convergence_traces)[2]):
        if np.all(np.argmax(att.convergence_traces[:,:,i], axis=0) == knownkey):
            return (i+1)*att.convergence_step
    return -1

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
from cycler import cycler

def plot_convergance(ax, att, success = None, ylim=[0.01, 0.08], nticks=10):
    tcol = [(1,0,0,1.0)]*16

    conv_shape = np.shape(att.convergence_traces[0])

    for bindex in range(0, 16):
        tc = tcol[bindex]
        ax.set_prop_cycle(cycler('color', [(1,0,0,0) if i == knownkey[bindex] else (.5,.5,.5, 0.2) for i in range(0,256)]))
        ax.plot(att.convergence_traces[:, bindex,:].T,)

    for bindex in range(0, 16):
        tc = tcol[bindex]
        #ax.set_prop_cycle(cycler('color', [tc if i == knownkey[bindex] else (.5,.5,.5, 0.05) for i in range(0,256)]))
        ax.plot(att.convergence_traces[knownkey[bindex], bindex, :].T, 'r')

    tickstep = int(conv_shape[1] / nticks)

    if success:
        ax.plot([success/att.convergence_step, success/att.convergence_step], ylim, 'k--')

    xticks = [j for j in range(0, conv_shape[1], tickstep)]
    #xticks.append(conv_shape[1])
    xlabels = [str(i * att.convergence_step) for i in xticks]
    ax.set_xticks(xticks, xlabels, rotation=-45)
    ax.set_ylim(ylim)

In [ ]:
def get_min_max(results, knownkey, byteindex=0):
    max1 = np.max(results[0:knownkey,byteindex,:].T, axis=1)
    max2 = np.max(results[(knownkey+1):,byteindex,:].T, axis=1)
    maxline = np.max(np.array([max1, max2]), axis=0)
    
    min1 = np.min(results[0:knownkey,byteindex,:].T, axis=1)
    min2 = np.min(results[(knownkey+1):,byteindex,:].T, axis=1)
    minline = np.min(np.array([min1, min2]), axis=0)
    return minline, maxline

In [ ]:
import numpy as np
att = np.load(data_dir + "/" + "opticalisolator_mixer_120mhz_sync_120msps_cpa.ets.npy", allow_pickle=True).item()
success = find_success(att, knownkey)
print(success)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))
plot_convergance(ax[0], att, success, ylim=[0.02, 0.20])

x = [1.0/120E6 * i * 1E6 for i in range(0, len(att.results[knownkey[0], [0]].T))]

start = 800
end = 3000

for i in range(0, 16):
    minline, maxline = get_min_max(att.results, knownkey[i], i)
    ax[1].plot(x[start:end], minline[start:end], color=(.5,.5,.5, 0.6))
    ax[1].plot(x[start:end], maxline[start:end], color=(.5,.5,.5, 0.6))

for i in range(0, 16):
    ax[1].plot(x[start:end], att.results[knownkey[i], [i]].T[start:end], 'r')
    
ax[0].set_xlabel(r"Trace Number", fontsize=13)
ax[0].set_ylabel(r"CPA Result", fontsize=13)

ax[1].set_xlabel(r"Time from Trigger ($\mu$S)", fontsize=13)
ax[1].set_ylabel("CPA Result (All Bytes)", fontsize=13)

fig.suptitle(r"Clock Output PM-CPA Results, Optical Isolator", fontsize=16)

plt.tight_layout()

plt.savefig("opticalpmcpa.png", dpi=300)
plt.savefig("opticalpmcpa.eps", dpi=300)
    
plt.show()